In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('data/spx_weeklies.csv')

In [3]:
print(data.columns)

Index(['secid', 'date', 'symbol', 'symbol_flag', 'exdate', 'last_date',
       'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'volume',
       'open_interest', 'impl_volatility', 'delta', 'gamma', 'vega', 'theta',
       'optionid', 'cfadj', 'am_settlement', 'contract_size', 'ss_flag',
       'forward_price', 'expiry_indicator', 'root', 'suffix', 'cusip',
       'ticker', 'sic', 'index_flag', 'exchange_d', 'class', 'issue_type',
       'industry_group', 'issuer', 'div_convention', 'exercise_style',
       'am_set_flag'],
      dtype='object')


In [4]:
print(data.shape)

(3205352, 38)


In [5]:
print(data.iloc[321510])

secid                            108105
date                         21/12/2015
symbol              SPXW 151224P1000000
symbol_flag                           1
exdate                       24/12/2015
last_date                    14/12/2015
cp_flag                               P
strike_price                    1000000
best_bid                            0.0
best_offer                         0.05
volume                                0
open_interest                       851
impl_volatility                2.280087
delta                         -0.000227
gamma                          0.000002
vega                           0.156078
theta                         -21.65537
optionid                      109020381
cfadj                                 1
am_settlement                         0
contract_size                       100
ss_flag                               0
forward_price                       NaN
expiry_indicator                      w
root                                NaN


In [6]:
no_info = []

for column in data.columns.tolist():
    print(column)
    print(data[column].unique())
    print()

    if(len(data[column].unique())==1):
        no_info.append(column)

print(no_info)

secid
[108105]

date
['03/01/2012' '04/01/2012' '05/01/2012' ... '24/02/2023' '27/02/2023'
 '28/02/2023']

symbol
['SPXW 120106C1000000' 'SPXW 120106C1025000' 'SPXW 120106C1050000' ...
 'SPXW 230307P5000000' 'SPXW 230307P5200000' 'SPXW 230307P5400000']

symbol_flag
[1]

exdate
['06/01/2012' '13/01/2012' '21/01/2012' ... '03/03/2023' '06/03/2023'
 '07/03/2023']

last_date
[nan '29/12/2011' '30/12/2011' ... '27/02/2023' '26/02/2023' '28/02/2023']

cp_flag
['C' 'P']

strike_price
[1000000 1025000 1050000 1075000 1100000 1125000 1130000 1135000 1140000
 1145000 1150000 1155000 1160000 1165000 1170000 1175000 1180000 1185000
 1190000 1195000 1200000 1205000 1210000 1215000 1220000 1225000 1230000
 1235000 1240000 1245000 1250000 1255000 1260000 1265000 1270000 1275000
 1280000 1285000 1290000 1295000 1300000 1305000 1310000 1315000 1320000
 1325000 1330000 1335000 1340000 1345000 1350000 1375000 1400000 1425000
 1005000 1010000 1015000 1020000 1030000 1035000 1040000 1045000 1055000
 106000

In [7]:
data = data.drop(columns=no_info)
print(data.columns)

Index(['date', 'symbol', 'exdate', 'last_date', 'cp_flag', 'strike_price',
       'best_bid', 'best_offer', 'volume', 'open_interest', 'impl_volatility',
       'delta', 'gamma', 'vega', 'theta', 'optionid', 'am_settlement',
       'expiry_indicator'],
      dtype='object')


In [8]:
print(data.iloc[10000]['date'])

28/03/2012


In [9]:
print(data['date'].unique())

['03/01/2012' '04/01/2012' '05/01/2012' ... '24/02/2023' '27/02/2023'
 '28/02/2023']


In [10]:
print(data.shape)

(3205352, 18)
